In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
df = pd.read_csv('/content/preprocessed_DNN (5).csv', low_memory=False)
df

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,...,dns.qry.name.len-1.debian.pool.ntp.org,dns.qry.name.len-2.debian.pool.ntp.org,dns.qry.name.len-3.debian.pool.ntp.org,dns.qry.name.len-_googlecast._tcp.local,mqtt.conack.flags-0,mqtt.conack.flags-0x00000000,mqtt.protoname-0,mqtt.protoname-MQTT,mqtt.topic-0,mqtt.topic-Temperature_and_Humidity
0,0,0,0,0,0,0,0,0,110218025,1249069932,...,False,False,False,False,True,False,True,False,True,False
1,0,0,0,0,0,0,0,0,1,3306662301,...,False,False,False,False,True,False,True,False,True,False
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,True,False,True,False
3,0,0,0,0,0,0,0,0,930,142931531,...,False,False,False,False,True,False,True,False,True,False
4,0,0,0,0,0,0,0,0,442,984983596,...,False,False,False,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46578,0,0,0,0,0,0,0,0,15,1811671929,...,False,False,False,False,True,False,True,False,True,False
46579,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,True,False,True,False
46580,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,True,False,True,False
46581,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,True,False,True,False,True,False


In [ ]:
feat_cols = list(df.columns)
label_col = "Attack_type"

feat_cols.remove(label_col)

In [ ]:
skip_list = ["icmp.unused", "http.tls_port", "dns.qry.type", "mqtt.msg_decoded_as"]

In [ ]:
df[skip_list[3]].value_counts()

,count
mqtt.msg_decoded_as,
0,46583


In [ ]:
X = df.drop([label_col], axis=1)
y = df[label_col]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

del X
del y

In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_train =  label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
X_train =  min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [ ]:
#X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(37266, 68)
(9317, 68)


In [ ]:
input_shape = X_train.shape[1:]

In [ ]:
print(X_train.shape, X_test.shape)
print(input_shape)

(37266, 68) (9317, 68)
(68,)


In [ ]:
num_classes = len(np.unique(y_train))
num_classes

15

In [ ]:
from  tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [ ]:
print(y_train.shape, y_test.shape)

(37266, 15) (9317, 15)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, MultiHeadAttention,
    LayerNormalization, Add, Dropout, Dense, GlobalAveragePooling1D, Reshape
)

# Data shape:
num_features = 68
num_classes = 15

# For Conv1D, reshape input to (sequence_length, num_channels)
# Here, treat features as "sequence_length = 68", with 1 channel per feature
inputs = Input(shape=(num_features, 1))  # reshape your data to (batch, 68, 1)

# CNN feature extractor
x = Conv1D(32, 3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(2)(x)

x = Conv1D(64, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(2)(x)

# Transformer block
attention_output = MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
attention_output = Dropout(0.1)(attention_output)
x = Add()([x, attention_output])
x = LayerNormalization(epsilon=1e-6)(x)

ffn = tf.keras.Sequential([
    Dense(128, activation='relu'),
    Dense(x.shape[-1])
])
ffn_output = ffn(x)
x = Add()([x, ffn_output])
x = LayerNormalization(epsilon=1e-6)(x)

# Classification head
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 68, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 68, 32)    │        128 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 34, 32)    │          0 │ conv1d[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 34, 64)    │      6,208 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 17, 64)    │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 17, 64)    │     16,640 │ max_pooling1d_1[… │
│ (MultiHeadAttentio… │                   │            │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 17, 64)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 17, 64)    │          0 │ max_pooling1d_1[… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 17, 64)    │        128 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 17, 64)    │     16,576 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 17, 64)    │          0 │ layer_normalizat… │
│                     │                   │            │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 17, 64)    │        128 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      4,160 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 15)        │        975 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 44,943 (175.56 KB)

 Trainable params: 44,943 (175.56 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # or 'sparse_categorical_crossentropy'
    metrics=['accuracy']
)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from livelossplot import PlotLossesKeras


#model_weights_file_path = "model_best_weights.h5"
#checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min", save_weights_only=True)
#checkpoint = ModelCheckpoint(filepath=model_weights_file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min", save_weights_only=True)
early_stopping = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, mode="min", verbose=1, min_lr=0)
#plotlosses = PlotLossesKeras()
call_backs = [ early_stopping, lr_reduce]
EPOCHS = 15
BATCH_SIZE = 256
call_backs = [early_stopping, lr_reduce]
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    validation_split=0.1,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=call_backs,
                    #class_weight=class_weights,
                    verbose=1)

Epoch 1/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 21s 100ms/step - accuracy: 0.2348 - loss: nan - val_accuracy: 0.0630 - val_loss: nan - learning_rate: 0.0010
Epoch 2/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 21s 101ms/step - accuracy: 0.0613 - loss: nan - val_accuracy: 0.0630 - val_loss: nan - learning_rate: 0.0010
Epoch 3/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 20s 99ms/step - accuracy: 0.0632 - loss: nan - val_accuracy: 0.0630 - val_loss: nan - learning_rate: 0.0010
Epoch 4/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 20s 99ms/step - accuracy: 0.0607 - loss: nan - val_accuracy: 0.0630 - val_loss: nan - learning_rate: 0.0010
Epoch 5/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.0652 - loss: nan
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
146/146 ━━━━━━━━━━━━━━━━━━━━ 22s 110ms/step - accuracy: 0.0652 - loss: nan - val_accuracy: 0.0630 - val_loss: nan - learning_rate: 0.0010
Epoch 6/15
146/146 ━━━━━━━━━━━━━━━━━━━━ 19s 100ms/step - accuracy: 0.0603 - loss: nan - val_accuracy: 0.0630